In [3]:
import os
import sys
import numpy as np
import matplotlib
matplotlib.use('nbagg')
#from matplotlib import style
#style.use('ggplot')
import matplotlib.pyplot as plt

from skimage import feature
from scipy import stats, ndimage
from scipy.misc import factorial as fac

import pandas as pd

import photutils
import opticspy

from astropy import units as u
from astropy import stats
from astropy import visualization
from astropy import coordinates
from astropy.table import Table, vstack
from astropy.io import ascii, fits
from astropy.modeling import models, fitting, custom_model

In [2]:
ACT = 104
NODE = 3222
BRAD = 3228.5

In [4]:
NODE*ACT

335088

In [39]:
# for each node, the influence from each actuator
surf2act = np.fromfile("Surf2ActTEL_32.bin", dtype=np.float32).reshape(NODE, ACT)
nodecoor = ascii.read("bcv_node_coordinates.dat", names=["bcv_id", "bcv_x", "bcv_y", "bcv_z"])
for ax in ["bcv_x", "bcv_y"]:
    nodecoor[ax] /= BRAD
nodecoor['bcv_rho'] = np.sqrt(nodecoor['bcv_x']**2 + nodecoor['bcv_y']**2)
nodecoor['bcv_phi'] = np.arctan2(nodecoor['bcv_y'], nodecoor['bcv_x'])

In [40]:
nodecoor

bcv_id,bcv_x,bcv_y,bcv_z,bcv_rho,bcv_phi
int64,float64,float64,float64,float64,float64
5,0.0,-1.0,679.205,1.0,-1.57079632679
10,0.00696918073409,-0.999975220691,679.204,0.999999505738,-1.5638270862
15,0.0418336688865,-0.999123431934,679.204,0.999998844046,-1.52895039801
20,0.0936224252749,-0.995607867431,679.205,1.0000000921,-1.47703659865
25,0.113820659749,-0.993501626142,679.205,1.00000031187,-1.45672849841
31,0.161375251665,-0.986891745393,679.204,0.999998644486,-1.40871209415
43,0.0859284497445,-0.982282793867,670.313,0.98603406919,-1.48354012798
44,0.120300449125,-0.982282793867,672.585,0.989621991059,-1.44893290375
49,0.188877187548,-0.982000929224,679.205,1.00000020849,-1.38077773907


In [38]:
9/11*386.2

315.9818181818182

In [77]:
im = np.arange(90).reshape(9, 10)

In [78]:
np.indices(im.shape, dtype=float)

array([[[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
        [ 2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.],
        [ 3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.],
        [ 4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.],
        [ 5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.],
        [ 6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.],
        [ 7.,  7.,  7.,  7.,  7.,  7.,  7.,  7.,  7.,  7.],
        [ 8.,  8.,  8.,  8.,  8.,  8.,  8.,  8.,  8.,  8.]],

       [[ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9.],
        [ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9.],
        [ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9.],
        [ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9.],
        [ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9.],
        [ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9.],
        [ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9.],
        [ 0.,  1.,  2.,  3.,  4.,  5.,

In [79]:
slopes = (np.indices(im.shape, dtype=float)/(np.r_[im.shape].reshape(-1, 1, 1))).reshape(2, -1)
slopes = np.vstack([slopes, np.ones(slopes.shape[1])])
slopes

array([[ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.11111111,  0.11111111,  0.11111111,  0.11111111,  0.11111111,
         0.11111111,  0.11111111,  0.11111111,  0.11111111,  0.11111111,
         0.22222222,  0.22222222,  0.22222222,  0.22222222,  0.22222222,
         0.22222222,  0.22222222,  0.22222222,  0.22222222,  0.22222222,
         0.33333333,  0.33333333,  0.33333333,  0.33333333,  0.33333333,
         0.33333333,  0.33333333,  0.33333333,  0.33333333,  0.33333333,
         0.44444444,  0.44444444,  0.44444444,  0.44444444,  0.44444444,
         0.44444444,  0.44444444,  0.44444444,  0.44444444,  0.44444444,
         0.55555556,  0.55555556,  0.55555556,  0.55555556,  0.55555556,
         0.55555556,  0.55555556,  0.55555556,  0.55555556,  0.55555556,
         0.66666667,  0.66666667,  0.66666667,  0.66666667,  0.66666667,
         0.66666667,  0.66666667,  0.66666667,  0.6

In [80]:
pinv = np.linalg.pinv(slopes)

In [81]:
np.dot(im.reshape(1, -1), pinv).ravel()[:2]

array([ 90.,  10.])

In [82]:
im

array([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9],
       [10, 11, 12, 13, 14, 15, 16, 17, 18, 19],
       [20, 21, 22, 23, 24, 25, 26, 27, 28, 29],
       [30, 31, 32, 33, 34, 35, 36, 37, 38, 39],
       [40, 41, 42, 43, 44, 45, 46, 47, 48, 49],
       [50, 51, 52, 53, 54, 55, 56, 57, 58, 59],
       [60, 61, 62, 63, 64, 65, 66, 67, 68, 69],
       [70, 71, 72, 73, 74, 75, 76, 77, 78, 79],
       [80, 81, 82, 83, 84, 85, 86, 87, 88, 89]])